In [2]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

Load data

In [4]:
data = loadmat('portfolio.mat')
volume = data['train_vol']
high = data['train_hi']
low = data['train_lo']
close = data['train_cl']

In [8]:
print data.keys()

['train_vol', 'train_stocks', '__header__', '__globals__', 'train_op', 'train_lo', 'train_cl', '__version__', 'train_hi']


In [10]:
data['train_op']

array([[  36.1 ,   32.58,   70.79, ...,   23.78,   74.9 ,   62.14],
       [  36.15,   32.16,   67.85, ...,   23.55,   73.61,   62.62],
       [  35.11,   29.68,   67.37, ...,   23.77,   73.66,   61.96],
       ..., 
       [  46.6 ,   16.05,  333.1 , ...,   49.44,   24.49,   61.34],
       [  47.37,   16.59,  343.51, ...,   50.38,   24.05,   62.59],
       [  48.82,   16.42,  355.  , ...,   53.62,   23.52,   61.69]])